In [1]:
#!pip install regex, demoji, plotly, wordcloud
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re

import regex
import demoji

import numpy as np
from collections import Counter

import plotly.express as px
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

### Paso 1: Definir funciones necesarias

In [3]:
# Patron regex para identificar el comienzo de cada línea del txt con la fecha y la hora
def IniciaConFechaYHora(s):
    # Nuevo patrón para fechas con coma y hora en formato 24h
    # Ejemplo: '27/5/2025, 13:49 - '
    patron = r'^\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{2} -'
    return bool(re.match(patron, s))

# Patrón para encontrar a los miembros del grupo dentro del txt
def EncontrarMiembro(s):
    patrones = ['林尚禾🌌:','親愛的❤️:']

    patron = '^' + '|'.join(patrones)
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de miembro
    if resultado:
        return True
    return False

# Separar las partes de cada línea del txt: Fecha, Hora, Miembro y Mensaje
def ObtenerPartes(linea):
    splitLinea = linea.split(' - ', 1)
    FechaHora = splitLinea[0]                     # '27/5/2025, 13:49'
    splitFechaHora = FechaHora.split(', ')
    Fecha = splitFechaHora[0]                     # '27/5/2025'
    Hora = splitFechaHora[1]                      # '13:49'
    Mensaje = splitLinea[1] if len(splitLinea) > 1 else ''
    if EncontrarMiembro(Mensaje):
        splitMensaje = Mensaje.split(': ', 1)
        if len(splitMensaje) == 2:
            Miembro, Mensaje = splitMensaje
        else:
            Miembro = None
    else:
        Miembro = None
    return Fecha, Hora, Miembro, Mensaje

### Paso 2: Obtener el dataframe usando el archivo txt y las funciones definidas

In [4]:
# Leer el archivo txt descargado del chat de WhatsApp
RutaChat = '/content/Chat de WhatsApp con Estela ✨.txt'

# Lista para almacenar los datos (Fecha, Hora, Miembro, Mensaje) de cada línea del txt
DatosLista = []
with open(RutaChat, encoding="utf-8") as fp:
    fp.readline() # Eliminar primera fila relacionada al cifrado de extremo a extremo
    Fecha, Hora, Miembro = None, None, None
    while True:
        linea = fp.readline()
        if not linea:
            break
        linea = linea.strip()
        if IniciaConFechaYHora(linea): # Si cada línea del txt coincide con el patrón fecha y hora
            Fecha, Hora, Miembro, Mensaje = ObtenerPartes(linea) # Obtener datos de cada línea del txt
            DatosLista.append([Fecha, Hora, Miembro, Mensaje])

# Convertir la lista con los datos a dataframe
df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje'])

# Cambiar la columna Fecha a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%Y")

# Eliminar los posibles campos vacíos del dataframe
# y lo que no son mensajes como cambiar el asunto del grupo o agregar a alguien
df = df.dropna()

# Resetear el índice
df.reset_index(drop=True, inplace=True)
df

,Fecha,Hora,Miembro,Mensaje
0,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
1,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
2,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
3,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
4,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
...,...,...,...,...
35941,2025-10-10,09:50,林尚禾🌌,<Multimedia omitido>
35942,2025-10-10,10:03,林尚禾🌌,Cualquier cosa me consultan
35943,2025-10-10,10:03,林尚禾🌌,<Multimedia omitido>
35944,2025-10-10,10:03,林尚禾🌌,<Multimedia omitido>


#### Filtrar el chat por fecha de acuerdo a lo requerido

In [5]:
start_date = '2024-09-01'
end_date = '2025-08-01'

df = df[(df['Fecha'] >= start_date) & (df['Fecha'] <= end_date)]
df

,Fecha,Hora,Miembro,Mensaje
0,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
1,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
2,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
3,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
4,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>
...,...,...,...,...
33704,2025-08-01,23:54,林尚禾🌌,"Si, varias"
33705,2025-08-01,23:54,林尚禾🌌,Siii jjaja
33706,2025-08-01,23:55,林尚禾🌌,Segurísimo pues
33707,2025-08-01,23:56,林尚禾🌌,Jajajaj


### Paso 3: Estadísticas de mensajes, multimedia, emojis y links

#### Total de mensajes, multimedia, emojis y links enviados

In [6]:
def ObtenerEmojis(Mensaje):
    emoji_lista = []
    data = regex.findall(r'\X', Mensaje)  # Obtener lista de caracteres de cada mensaje
    for caracter in data:
        if demoji.replace(caracter) != caracter:
            emoji_lista.append(caracter)
    return emoji_lista

# Obtener la cantidad total de mensajes
total_mensajes = df.shape[0]

# Obtener la cantidad de archivos multimedia enviados
multimedia_mensajes = df[df['Mensaje'] == '<Multimedia omitido>'].shape[0]

# Obtener la cantidad de emojis enviados
df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis) # Se agrega columna 'Emojis'
emojis = sum(df['Emojis'].str.len())

# Obtener la cantidad de links enviados
url_patron = r'(https?://\S+)'
df['URLs'] = df.Mensaje.apply(lambda x: len(re.findall(url_patron, x))) # Se agrega columna 'URLs'
links = sum(df['URLs'])

# Obtener la cantidad de encuestas
encuestas = df[df['Mensaje'] == 'POLL:'].shape[0]

# Todos los datos pasarlo a diccionario
estadistica_dict = {'Tipo': ['Mensajes', 'Multimedia', 'Emojis', 'Links', 'Encuestas'],
        'Cantidad': [total_mensajes, multimedia_mensajes, emojis, links, encuestas]
        }

#Convertir diccionario a dataframe
estadistica_df = pd.DataFrame(estadistica_dict, columns = ['Tipo', 'Cantidad'])

# Establecer la columna Tipo como índice
estadistica_df = estadistica_df.set_index('Tipo')
estadistica_df

/tmp/ipython-input-1746415161.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis) # Se agrega columna 'Emojis'
/tmp/ipython-input-1746415161.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['URLs'] = df.Mensaje.apply(lambda x: len(re.findall(url_patron, x))) # Se agrega columna 'URLs'


,Cantidad
Tipo,
Mensajes,33709
Multimedia,6099
Emojis,858
Links,103
Encuestas,0


#### Emojis más usados

In [7]:
# Obtener emojis más usados y las cantidades en el chat del grupo del dataframe
emojis_lista = list([a for b in df.Emojis for a in b])
emoji_diccionario = dict(Counter(emojis_lista))
emoji_diccionario = sorted(emoji_diccionario.items(), key=lambda x: x[1], reverse=True)

# Convertir el diccionario a dataframe
emoji_df = pd.DataFrame(emoji_diccionario, columns=['Emoji', 'Cantidad'])

# Establecer la columna Emoji como índice
emoji_df = emoji_df.set_index('Emoji').head(10)

print('Número emojis únicos usados: ', len(emoji_df), '\n')
emoji_df

Número emojis únicos usados:  10 



,Cantidad
Emoji,
😌,125
👀,86
😔,78
😝,73
💀,45
😎,43
👌,37
🧐,36
🤝,34


In [8]:
# Plotear el pie de los emojis más usados
fig = px.pie(emoji_df, values='Cantidad', names=emoji_df.index, hole=.3, template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20)
fig.update_layout(title={'text': '🤗 Emojis que más usamos', 'y':0.96, 'x':0.5, 'xanchor': 'center'}, font=dict(size=17))
fig.show()

### Paso 4: Estadísticas de los miembros del grupo

#### Miembros más activos

In [9]:
# Determinar los miembros más activos del grupo
df_MiembrosActivos = df.groupby('Miembro')['Mensaje'].count().sort_values(ascending=False).to_frame()
df_MiembrosActivos.reset_index(inplace=True)
df_MiembrosActivos.index = np.arange(1, len(df_MiembrosActivos)+1)
df_MiembrosActivos['% Mensaje'] = (df_MiembrosActivos['Mensaje'] / df_MiembrosActivos['Mensaje'].sum()) * 100
df_MiembrosActivos

,Miembro,Mensaje,% Mensaje
1,林尚禾🌌,33709,100.0


#### Estadísticas por miembro

In [10]:
# Separar mensajes (sin multimedia) y multimedia (stickers, fotos, videos)
multimedia_df = df[df['Mensaje'] == '<Media omitted>']
mensajes_df = df.drop(multimedia_df.index)

# Contar la cantidad de palabras y letras por mensaje
mensajes_df['Letras'] = mensajes_df['Mensaje'].apply(lambda s : len(s))
mensajes_df['Palabras'] = mensajes_df['Mensaje'].apply(lambda s : len(s.split(' ')))
mensajes_df.tail()

,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,Letras,Palabras
33704,2025-08-01,23:54,林尚禾🌌,"Si, varias",[],0,10,2
33705,2025-08-01,23:54,林尚禾🌌,Siii jjaja,[],0,10,2
33706,2025-08-01,23:55,林尚禾🌌,Segurísimo pues,[],0,15,2
33707,2025-08-01,23:56,林尚禾🌌,Jajajaj,[],0,7,1
33708,2025-08-01,23:57,林尚禾🌌,La que es hija de mi tía,[],0,24,7


In [11]:
# Obtener a todos los miembros
miembros = mensajes_df.Miembro.unique()

# Crear diccionario donde se almacenará todos los datos
dictionario = {}

for i in range(len(miembros)):
    lista = []
    # Filtrar mensajes de un miembro en específico
    miembro_df= mensajes_df[mensajes_df['Miembro'] == miembros[i]]

    # Agregar a la lista el número total de mensajes enviados
    lista.append(miembro_df.shape[0])

    # Agregar a la lista el número de palabras por total de mensajes (palabras por mensaje)
    palabras_por_msj = (np.sum(miembro_df['Palabras']))/miembro_df.shape[0]
    lista.append(palabras_por_msj)

    # Agregar a la lista el número de mensajes multimedia enviados
    multimedia = multimedia_df[multimedia_df['Miembro'] == miembros[i]].shape[0]
    lista.append(multimedia)

    # Agregar a la lista el número total de emojis enviados
    emojis = sum(miembro_df['Emojis'].str.len())
    lista.append(emojis)

    # Agregar a la lista el número total de links enviados
    links = sum(miembro_df['URLs'])
    lista.append(links)

    # Asignar la lista como valor a la llave del diccionario
    dictionario[miembros[i]] = lista

print(dictionario)

{'林尚禾🌌': [33709, np.float64(5.59725889228396), 0, 858, 103]}


In [12]:
# Convertir de diccionario a dataframe
miembro_stats_df = pd.DataFrame.from_dict(dictionario)

# Cambiar el índice por la columna agregada 'Estadísticas'
estadísticas = ['Mensajes', 'Palabras por mensaje', 'Multimedia', 'Emojis', 'Links']
miembro_stats_df['Estadísticas'] = estadísticas
miembro_stats_df.set_index('Estadísticas', inplace=True)

# Transponer el dataframe
miembro_stats_df = miembro_stats_df.T

#Convertir a integer las columnas Mensajes, Multimedia Emojis y Links
miembro_stats_df['Mensajes'] = miembro_stats_df['Mensajes'].apply(int)
miembro_stats_df['Multimedia'] = miembro_stats_df['Multimedia'].apply(int)
miembro_stats_df['Emojis'] = miembro_stats_df['Emojis'].apply(int)
miembro_stats_df['Links'] = miembro_stats_df['Links'].apply(int)
miembro_stats_df = miembro_stats_df.sort_values(by=['Mensajes'], ascending=False)
miembro_stats_df

Estadísticas,Mensajes,Palabras por mensaje,Multimedia,Emojis,Links
林尚禾🌌,33709,5.597259,0,858,103


### Paso 5: Estadísticas del comportamiento del grupo

In [13]:
df['rangoHora'] = pd.to_datetime(df['Hora'], format='%H:%M')


# Define a function to create the "Range Hour" column
def create_range_hour(hour):
    hour = pd.to_datetime(hour)  # Convertir a objeto de Python datetime si es necesario
    start_hour = hour.hour
    end_hour = (hour + pd.Timedelta(hours=1)).hour
    return f'{start_hour:02d} - {end_hour:02d} h'

# # Apply the function to create the "Range Hour" column
df['rangoHora'] = df['rangoHora'].apply(create_range_hour)
df

/tmp/ipython-input-9566059.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-9566059.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,rangoHora
0,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h
1,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h
2,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h
3,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h
4,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h
...,...,...,...,...,...,...,...
33704,2025-08-01,23:54,林尚禾🌌,"Si, varias",[],0,23 - 00 h
33705,2025-08-01,23:54,林尚禾🌌,Siii jjaja,[],0,23 - 00 h
33706,2025-08-01,23:55,林尚禾🌌,Segurísimo pues,[],0,23 - 00 h
33707,2025-08-01,23:56,林尚禾🌌,Jajajaj,[],0,23 - 00 h


In [14]:
df['DiaSemana'] = df['Fecha'].dt.strftime('%A')
mapeo_dias_espanol = {'Monday': '1 Lunes','Tuesday': '2 Martes','Wednesday': '3 Miércoles','Thursday': '4 Jueves',
                      'Friday': '5 Viernes','Saturday': '6 Sábado','Sunday': '7 Domingo'}
df['DiaSemana'] = df['DiaSemana'].map(mapeo_dias_espanol)
df

/tmp/ipython-input-2757418839.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-2757418839.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,rangoHora,DiaSemana
0,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h,4 Jueves
1,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h,4 Jueves
2,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h,4 Jueves
3,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h,4 Jueves
4,2024-09-05,12:55,林尚禾🌌,<Multimedia omitido>,[],0,12 - 13 h,4 Jueves
...,...,...,...,...,...,...,...,...
33704,2025-08-01,23:54,林尚禾🌌,"Si, varias",[],0,23 - 00 h,5 Viernes
33705,2025-08-01,23:54,林尚禾🌌,Siii jjaja,[],0,23 - 00 h,5 Viernes
33706,2025-08-01,23:55,林尚禾🌌,Segurísimo pues,[],0,23 - 00 h,5 Viernes
33707,2025-08-01,23:56,林尚禾🌌,Jajajaj,[],0,23 - 00 h,5 Viernes


#### Número de mensajes por rango de hora

In [15]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por hora'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
mensajes_hora = df.groupby('rangoHora').count().reset_index()

# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(mensajes_hora, x='rangoHora', y='# Mensajes por hora', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
fig.update_layout(
    title={'text': 'Mensajes con ella ❤️ por hora', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
    font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Rango de hora', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

/tmp/ipython-input-3375260800.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Número de mensajes por día

In [16]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por día'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
date_df = df.groupby('DiaSemana').count().reset_index()


# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(date_df, x='DiaSemana', y='# Mensajes por día', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
fig.update_layout(
    title={'text': 'Mensajes con ella ❤️ por día', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
    font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Día', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

/tmp/ipython-input-239596912.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Número de mensajes a través del tiempo

In [17]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por día'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
date_df = df.groupby('Fecha').sum().reset_index()

# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(date_df, x='Fecha', y='# Mensajes por día', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
fig.update_layout(
    title={'text': 'Mensajes con ella ❤️ a lo largo del tiempo', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
    font=dict(size=17))
fig.update_xaxes(title_text='Fecha', tickangle=45, nticks=35)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

/tmp/ipython-input-2885617712.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Word Cloud de palabras más usadas

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import string

# Copia local de stopwords, sin modificar la original
custom_stopwords = set(STOPWORDS)
custom_stopwords.update([
    'que', 'qué', 'con', 'de', 'te', 'en', 'la', 'lo', 'le', 'el', 'las', 'los', 'les', 'por',
    'es', 'son', 'se', 'para', 'un', 'una', 'chicos', 'su', 'si', 'chic', 'nos', 'ya', 'hay',
    'esta', 'está', 'pero', 'del', 'mas', 'más', 'eso', 'este', 'como', 'así', 'todo',
    'https', 'multimedia', 'omitido', 'y', 'mi', 'o', 'q', 'yo', 'al', 'vos', 'jajaja', 'jajajaj',
    'porque', 'sip', 'también', 'entonces', 'después', 'voy', 'uh', 'ay', 'ja', 'jaja',
    'jaj', 'jeje', 'ajaja', 're', 'no', 'sí', 'a', 'eh', 'uy', 'jajajaja', 'va', 'igual', 'ahora', 'mensaje', 'editó'
])

# Limpieza de puntuación
def limpiar_palabras(texto):
    texto = str(texto).lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))  # elimina puntuación
    return texto.split()

# Usar join en vez de sumar strings en bucle
todas_las_palabras = []
for mensaje in mensajes_df['Mensaje']:
    palabras = limpiar_palabras(mensaje)
    todas_las_palabras.extend(palabras)

# Generar string total
texto_total = ' '.join(todas_las_palabras)

# Cargar la máscara
mask = np.array(Image.open('/content/heart.jpg'))

# Generar la nube
wordcloud = WordCloud(
    width=800,
    height=800,
    background_color='black',
    stopwords=custom_stopwords,
    max_words=100,
    min_font_size=5,
    mask=mask,
    colormap='OrRd'
).generate(texto_total)

# Mostrar la imagen
wordcloud.to_image()

FileNotFoundError: [Errno 2] No such file or directory: '/content/heart.jpg'

In [ ]:
palabras_romanticas = [
    'amor', 'te amo', 'te quiero', 'mi amor', 'cariño', 'princesa', 'corazón',
    'reina', 'te extraño', 'te adoro', 'mi vida', 'preciosa', 'hermosa',
    'linda', 'bonita', 'besito', 'besos', 'abrazos'
]
from collections import Counter

contador_romantico = Counter()

for mensaje in df['Mensaje'].dropna():
    mensaje = str(mensaje).lower()
    for palabra in palabras_romanticas:
        if palabra in mensaje:
            contador_romantico[palabra] += mensaje.count(palabra)

# Mostrar los más comunes
contador_romantico.most_common(10)
import matplotlib.pyplot as plt

# Separar claves y valores
palabras = list(contador_romantico.keys())
frecuencias = list(contador_romantico.values())

plt.figure(figsize=(10,5))
plt.bar(palabras, frecuencias, color='lightcoral')
plt.title('Palabras románticas más usadas ❤️')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
